In [2]:
#IMportamos librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd

In [3]:
#Se lee pagina de los planes de protección de aves
planes = pd.read_excel('../data_raw/Catalogacion_Especies_Aves_y_Planes_accion_12-2022.xlsx', sheet_name = 'SPS-Planes' )
planes

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,EXT,GAL,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea
0,Abejaruco europeo,Merops apiaster,NaN,NaN,NaN,NaN,PC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abejero europeo,Pernis apivorus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abubilla común,Upupa epops,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Agachadiza común,Gallinago gallinago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NaN,NaN,NaN,NaN,NaN,NaN,...,PR-2005,NaN,PR,NaN,NaN,NaN,NaN,SAP-2008,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,Zampullín común,Tachybaptus ruficollis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,PC,NaN,NaN,NaN,NaN,NaN
167,Zampullín cuellinegro,Podiceps nigricollis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PCH,NaN,NaN,NaN,PCH,NaN,NaN,NaN,NaN,NaN
168,Zarapito fino,Numenius tenuirostris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAP-1996,NaN,NaN


In [4]:
planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Nombre común                                         170 non-null    object
 1   Nombre científico                                    170 non-null    object
 2   ESTRATEGIA ESTATAL                                   51 non-null     object
 3   AND                                                  35 non-null     object
 4   ARA                                                  23 non-null     object
 5   AST                                                  19 non-null     object
 6   BAL                                                  21 non-null     object
 7   CANA                                                 24 non-null     object
 8   CANT                                                 14 non-null     object
 9  

In [5]:
planes.isnull().sum()

Nombre común                                             0
Nombre científico                                        0
ESTRATEGIA ESTATAL                                     119
AND                                                    135
ARA                                                    147
AST                                                    151
BAL                                                    149
CANA                                                   146
CANT                                                   156
CAT                                                    106
CLM                                                    109
CYL                                                    155
EUSK                                                   142
EXT                                                    117
GAL                                                    140
MAD                                                    143
MUR                                                    1

#### Hay muchísimos nulos, eso se debe a que solo hacen falta planes en algunas comunidades.

## Arreglamos algunos datos

#### Hay un par de casillas que están coloreadas pero no tienen texto, las rellenamos

In [6]:
# Casilla roja sin texto
planes['Nombre común'][166]
planes['EXT'][166]

nan

In [7]:
# La rellenamos
planes['EXT'].loc[166]='PC'

In [8]:
# Comprobamos
planes['EXT'][166]

'PC'

In [9]:
# Casilla roja sin texto
print(planes['Nombre común'][18])
print(planes['CAT'][18])

Alcotán europeo
nan


In [10]:
# La rellenamos
planes['CAT'].loc[18]='PR'

In [11]:
# Comprobamos
planes['CAT'][18]

'PR'

## Buscar colores en excel con openpyxl

#### Necesitamos saber si una celda es verde o roja para saber si existe un plan ya aprobado o si hace falta un plan pero aún no está aprobado. Para eso usaremos la librería de python openpyxl.

In [12]:
# Cargamos la librería
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Color

In [13]:
# Cargamos el archivo ecel y seleccionamos la hoja que queremos leer, en este caso la de los planes.
wb = load_workbook('../data_raw/Catalogacion_Especies_Aves_y_Planes_accion_12-2022.xlsx', data_only=True)
sh = wb['SPS-Planes']

In [14]:
sh

<Worksheet "SPS-Planes">

In [15]:
# Vamos a comprobar cuál es el código de color de una celda verde con la función fill.start_color
verde = sh['C6'].fill.start_color.index
print('El código de color de las celdas verdes es ' f'{verde}')

El código de color de las celdas verdes es FF92D050


In [16]:
# Comprobamos también cuál es el código de color de una celda roja
rojo = sh['C7'].fill.start_color.index
print('El código de color de las celdas rojas es ' f'{rojo}')

El código de color de las celdas rojas es FFFF0000


In [17]:
# Definimos una función de prueba que busque en todo el dataframe celdas de un color específico
def buscaColor(excel, color_elegido):
    for row in excel.iter_rows():
        for cell in row:
            if cell.fill.fgColor.rgb == color_elegido.rgb:
                print(f"La celda {cell.coordinate} tiene el color rojo")


In [18]:
# La probamos
color_rojo = Color(rgb='FFFF0000') # Color rojo en formato RGB
color_verde = Color(rgb='FF92D050') # Color verde en formato RGB
buscaColor(sh, color_verde)

La celda C6 tiene el color rojo
La celda D6 tiene el color rojo
La celda K6 tiene el color rojo
La celda L6 tiene el color rojo
La celda N6 tiene el color rojo
La celda U6 tiene el color rojo
La celda E7 tiene el color rojo
La celda G7 tiene el color rojo
La celda K7 tiene el color rojo
La celda L7 tiene el color rojo
La celda M7 tiene el color rojo
La celda N7 tiene el color rojo
La celda Q7 tiene el color rojo
La celda R7 tiene el color rojo
La celda S7 tiene el color rojo
La celda U7 tiene el color rojo
La celda D8 tiene el color rojo
La celda G8 tiene el color rojo
La celda F9 tiene el color rojo
La celda C10 tiene el color rojo
La celda D10 tiene el color rojo
La celda S10 tiene el color rojo
La celda T11 tiene el color rojo
La celda V13 tiene el color rojo
La celda W13 tiene el color rojo
La celda D21 tiene el color rojo
La celda F21 tiene el color rojo
La celda G21 tiene el color rojo
La celda H21 tiene el color rojo
La celda S21 tiene el color rojo
La celda U21 tiene el color r

#### Vemos que funciona. Utilizaremos un proceso parecido para detectar si un plan está activo o si aún no se activó un plan

color_buscar = Color(rgb='00FF00') # Color verde en formato RGB
for row in sheet.iter_rows():
    for cell in row:
        if cell.fill.fgColor.rgb == color_buscar.rgb:
            print(f"La celda {cell.coordinate} tiene el color verde")

In [19]:
planes_color = planes[['Nombre común']]
planes_color['Color']=np.nan
planes_color.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nombre común  170 non-null    object 
 1   Color         0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


/tmp/ipykernel_5034/3980396965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planes_color['Color']=np.nan


In [20]:
# Creamos una lista para guardar los colores y recorremos el excel buscando si es verde o rojo
colores=[]
for row in sh.iter_rows():
        x=''
        for cell in row:
            if cell.fill.fgColor.rgb == color_rojo.rgb:
                x=x+',Rojo'
                #print(x)
            else:
                 pass
            if cell.fill.fgColor.rgb == color_verde.rgb:
                x=x+',Verde'
                #print(x)
            else:
                 pass
        #print(x)
        colores.append(x)
        #print(colores)
        x=''
colores
#planes_color['Color'] = planes_color.apply(colores)

['',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Verde,Verde,Verde,Verde,Verde,Rojo,Verde',
 ',Rojo,Rojo,Verde,Verde,Rojo,Verde,Verde,Verde,Verde,Rojo,Rojo,Verde,Verde,Verde,Rojo,Verde',
 ',Rojo,Verde,Rojo,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo',
 ',Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Verde',
 ',Rojo',
 ',Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo,Verde,Rojo,Verde,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo,Verde',
 ',Verde',
 ',Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde',
 ',Rojo',
 ',Verde,Verde',
 ',Rojo',
 ',Rojo,Verde',
 ',Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Verde,Rojo,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Verde',
 ',Verde,R

In [21]:
# Comprobamos que funcionó
colores

['',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Verde,Verde,Verde,Verde,Verde,Rojo,Verde',
 ',Rojo,Rojo,Verde,Verde,Rojo,Verde,Verde,Verde,Verde,Rojo,Rojo,Verde,Verde,Verde,Rojo,Verde',
 ',Rojo,Verde,Rojo,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo',
 ',Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Verde',
 ',Rojo',
 ',Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo,Verde,Rojo,Verde,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo,Verde',
 ',Verde',
 ',Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde',
 ',Rojo',
 ',Verde,Verde',
 ',Rojo',
 ',Rojo,Verde',
 ',Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Verde,Rojo,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Verde',
 ',Verde,R

In [22]:
cosa = []

In [23]:
coloritos = pd.DataFrame(colores, columns = ['HAY_PLAN'])
coloritos

,HAY_PLAN
0,
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Rojo"
...,...
309,
310,
311,
312,


In [24]:
coloritos.drop(0, axis=0, inplace=True)
coloritos

,HAY_PLAN
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Rojo"
5,",Verde,Verde,Verde,Verde,Verde,Rojo,Verde"
...,...
309,
310,
311,
312,


In [25]:
coloritos['HAY_PLAN'][170]

',Verde,Rojo,Verde'

In [26]:
colorsotes = coloritos.loc[:170]
colorsotes.reset_index(inplace=True)
colorsotes.drop(['index'], axis=1, inplace=True)
colorsotes

/tmp/ipykernel_5034/1411912946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colorsotes.drop(['index'], axis=1, inplace=True)


,HAY_PLAN
0,",Rojo"
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Verde,Verde,Verde,Verde,Verde,Rojo,Verde"
...,...
165,",Rojo"
166,",Rojo,Rojo"
167,",Rojo,Rojo,Rojo"
168,",Verde"


In [27]:
nombres = planes[['Nombre común']]
nombres

,Nombre común
0,Abejaruco europeo
1,Abejero europeo
2,Abubilla común
3,Agachadiza común
4,Águila imperial ibérica
...,...
165,Vencejo real
166,Zampullín común
167,Zampullín cuellinegro
168,Zarapito fino


In [28]:
planes_con_colores = pd.concat([nombres, colorsotes], axis=1)
planes_con_colores

,Nombre común,HAY_PLAN
0,Abejaruco europeo,",Rojo"
1,Abejero europeo,",Rojo"
2,Abubilla común,",Rojo"
3,Agachadiza común,",Rojo"
4,Águila imperial ibérica,",Verde,Verde,Verde,Verde,Verde,Rojo,Verde"
...,...,...
165,Vencejo real,",Rojo"
166,Zampullín común,",Rojo,Rojo"
167,Zampullín cuellinegro,",Rojo,Rojo,Rojo"
168,Zarapito fino,",Verde"


In [29]:
planes_con_colores["HAY_PLAN"].replace({'^\,':''}, regex=True, inplace=True)
planes_con_colores

,Nombre común,HAY_PLAN
0,Abejaruco europeo,Rojo
1,Abejero europeo,Rojo
2,Abubilla común,Rojo
3,Agachadiza común,Rojo
4,Águila imperial ibérica,"Verde,Verde,Verde,Verde,Verde,Rojo,Verde"
...,...,...
165,Vencejo real,Rojo
166,Zampullín común,"Rojo,Rojo"
167,Zampullín cuellinegro,"Rojo,Rojo,Rojo"
168,Zarapito fino,Verde


In [30]:
excel_prueba = sh

In [31]:
excel_prueba.delete_cols(3)
excel_prueba.delete_cols(21)
excel_prueba.delete_cols(22)
excel_prueba.delete_cols(23)

In [32]:
list_with_values=[]
for cell in excel_prueba[1]:
    list_with_values.append(cell.value)

list_with_values

['Nombre común',
 'Nombre científico',
 'AND',
 'ARA',
 'AST',
 'BAL',
 'CANA',
 'CANT',
 'CAT',
 'CLM',
 'CYL',
 'EUSK',
 'EXT',
 'GAL',
 'MAD',
 'MUR',
 'NAV',
 'RIO',
 'VAL',
 'Planes Acción Comisión Europea',
 'Planes Manejo Especies Cinegéticas Comisión Europea']

In [33]:
column_list = [cell.column for cell in excel_prueba[1]]
column_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [34]:
excel_prueba.delete_cols(20)
excel_prueba.delete_cols(21)
list_with_values=[]
for cell in excel_prueba[1]:
    list_with_values.append(cell.value)

list_with_values

['Nombre común',
 'Nombre científico',
 'AND',
 'ARA',
 'AST',
 'BAL',
 'CANA',
 'CANT',
 'CAT',
 'CLM',
 'CYL',
 'EUSK',
 'EXT',
 'GAL',
 'MAD',
 'MUR',
 'NAV',
 'RIO',
 'VAL',
 'Planes Manejo Especies Cinegéticas Comisión Europea']

In [35]:
column_list = [cell.column for cell in excel_prueba[1]]
column_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [36]:
excel_prueba.delete_cols(20)
list_with_values=[]
for cell in excel_prueba[1]:
    list_with_values.append(cell.value)

list_with_values

['Nombre común',
 'Nombre científico',
 'AND',
 'ARA',
 'AST',
 'BAL',
 'CANA',
 'CANT',
 'CAT',
 'CLM',
 'CYL',
 'EUSK',
 'EXT',
 'GAL',
 'MAD',
 'MUR',
 'NAV',
 'RIO',
 'VAL']

In [37]:
list_with_values=[]
for cell in excel_prueba[1]:
    list_with_values.append(cell.value)

list_with_values

['Nombre común',
 'Nombre científico',
 'AND',
 'ARA',
 'AST',
 'BAL',
 'CANA',
 'CANT',
 'CAT',
 'CLM',
 'CYL',
 'EUSK',
 'EXT',
 'GAL',
 'MAD',
 'MUR',
 'NAV',
 'RIO',
 'VAL']

In [38]:
colores_ccaa=[]
for row in excel_prueba.iter_rows():
        x=''
        for cell in row:
            if cell.fill.fgColor.rgb == color_rojo.rgb:
                x=x+',Rojo'
                #print(x)
            else:
                 pass
            if cell.fill.fgColor.rgb == color_verde.rgb:
                x=x+',Verde'
                #print(x)
            else:
                 pass
        #print(x)
        colores_ccaa.append(x)
        #print(colores)
        x=''
colores_ccaa

['',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Verde,Verde,Verde,Verde,Rojo',
 ',Rojo,Verde,Verde,Rojo,Verde,Verde,Verde,Verde,Rojo,Rojo,Verde,Verde,Verde,Rojo',
 ',Verde,Rojo,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo',
 ',Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 '',
 ',Rojo',
 ',Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Verde,Rojo,Verde,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde,Rojo',
 '',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo,Rojo,Rojo',
 '',
 ',Rojo',
 '',
 ',Rojo',
 ',Rojo',
 ',Rojo',
 ',Rojo,Rojo,Rojo,Rojo,Rojo',
 ',Verde,Rojo,Verde,Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Verde',
 ',Verde,Rojo,Verde,Rojo,Rojo,Rojo',
 ',Rojo,Rojo,Rojo',
 ',Verde,Rojo,Rojo,Rojo,Rojo,Rojo,Rojo,Verde',
 ',Rojo',
 ',Verde,Rojo',
 '',
 

In [39]:
colores_comunidades = pd.DataFrame(colores_ccaa, columns = ['HAY_PLAN'])
colores_comunidades

,HAY_PLAN
0,
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Rojo"
...,...
309,
310,
311,
312,


In [40]:
colores_comunidades.drop([0], axis=0, inplace=True)
colores_comunidades


,HAY_PLAN
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Rojo"
5,",Verde,Verde,Verde,Verde,Rojo"
...,...
309,
310,
311,
312,


In [41]:
colores_comunidades['HAY_PLAN'][170]

',Verde,Rojo'

In [42]:
colorsotes_comunidad = colores_comunidades.loc[:170]
colorsotes_comunidad.reset_index(drop=True, inplace=True)
colorsotes_comunidad

,HAY_PLAN
0,",Rojo"
1,",Rojo"
2,",Rojo"
3,",Rojo"
4,",Verde,Verde,Verde,Verde,Rojo"
...,...
165,",Rojo"
166,",Rojo,Rojo"
167,",Rojo,Rojo,Rojo"
168,


In [43]:
colorsotes_comunidad["HAY_PLAN"].replace({'^\,':''}, regex=True, inplace=True)
colorsotes_comunidad

/tmp/ipykernel_5034/1987603182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colorsotes_comunidad["HAY_PLAN"].replace({'^\,':''}, regex=True, inplace=True)


,HAY_PLAN
0,Rojo
1,Rojo
2,Rojo
3,Rojo
4,"Verde,Verde,Verde,Verde,Rojo"
...,...
165,Rojo
166,"Rojo,Rojo"
167,"Rojo,Rojo,Rojo"
168,


#### Ya tenemos la columna de los colores, más tarde la uniremos al dataset pincipal

In [44]:
# Guardamos por si acaso
#planes_con_colores.to_csv('planes_colores.csv')

## Extraer info de planes

#### Vamos a extraer la info de cada plan junto a la info de su lugar de aplicación (cca donde se aplica, estatal, europeo...)

In [45]:
planes

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,EXT,GAL,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea
0,Abejaruco europeo,Merops apiaster,NaN,NaN,NaN,NaN,PC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abejero europeo,Pernis apivorus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abubilla común,Upupa epops,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Agachadiza común,Gallinago gallinago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NaN,NaN,NaN,NaN,NaN,NaN,...,PR-2005,NaN,PR,NaN,NaN,NaN,NaN,SAP-2008,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,Zampullín común,Tachybaptus ruficollis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PC,NaN,NaN,NaN,PC,NaN,NaN,NaN,NaN,NaN
167,Zampullín cuellinegro,Podiceps nigricollis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PCH,NaN,NaN,NaN,PCH,NaN,NaN,NaN,NaN,NaN
168,Zarapito fino,Numenius tenuirostris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAP-1996,NaN,NaN


In [46]:
planes.fillna('NONECESITA', inplace=True)
planes

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,EXT,GAL,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR-2005,NONECESITA,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PC,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
166,Zampullín común,Tachybaptus ruficollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PC,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
167,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PCH,NONECESITA,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA
168,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA


In [47]:
# Creamos las 2 nuevas columnas
planes['Ubicacion']=np.nan
planes['Plan']=np.nan
# Recorremos las filas, añadiendo el nombre de las comunidades en la que el ave está presente a la columna 'Presente'
for index, row in planes.iterrows():
    x=''
    y=''
    if row['AND'] != 'NONECESITA':
        x = x + ',' + 'Andalucía'
        z = planes['AND'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['ARA'] != 'NONECESITA':
        x = x + ',' + 'Aragón'
        z = planes['ARA'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['AST'] != 'NONECESITA':
        x = x + ',' + 'Principado de Asturias'
        z = planes['AST'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['BAL'] != 'NONECESITA':
        x = x + ',' + 'Islas Baleares'
        z = planes['BAL'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['CANA'] != 'NONECESITA':
        x = x + ',' + 'Islas Canarias'
        z = planes['CANA'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['CANT'] != 'NONECESITA':
        x = x + ',' + 'Cantabria'
        z = planes['CANT'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['CAT'] != 'NONECESITA':
        x = x + ',' + 'Cataluña'
        z = planes['CAT'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['CLM'] != 'NONECESITA':
        x = x + ',' + 'Castilla-La Mancha'
        z = planes['CLM'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['CYL'] != 'NONECESITA':
        x = x + ',' + 'Castilla y León'
        z = planes['CYL'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['EUSK'] != 'NONECESITA':
        x = x + ',' + 'País Vasco'
        z = planes['EUSK'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['EXT'] != 'NONECESITA':
        x = x + ',' + 'Extremadura'
        z = planes['EXT'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['GAL'] != 'NONECESITA':
        x = x + ',' + 'Galicia'
        z = planes['GAL'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['MAD']  != 'NONECESITA':
        x = x + ',' + 'Comunidad de Madrid'
        z = planes['MAD'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['MUR'] != 'NONECESITA':
        x = x + ',' + 'Región de Murcia'
        z = planes['MUR'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['NAV'] != 'NONECESITA':
        x = x + ',' + 'Comunidad Foral de Navarra'
        z = planes['NAV'][index]
        y = y + ',' + z
        print(y)
        z = ''
    else:
        pass
    if row['RIO'] != 'NONECESITA':
        x = x + ',' + 'La Rioja'
        z = planes['RIO'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    if row['VAL'] != 'NONECESITA':
        x = x + ',' + 'Comunidad Valenciana'
        z = planes['VAL'][index]
        y = y + ',' + z
        z = ''
    else:
        pass
    #print(z)
    #print(index)
    planes['Ubicacion'][index] = x
    planes['Plan'][index] = y
    

planes

,PC,PR-2011(Modif. 2013),PRE-2009,PR,PR-2016,PC-2006,PG-2001,PCH-2005,PC,PR,PR-2012/PR-2016,PR-1996
,PC-2001,PC,PC,PC,PC,PR,PCH,PC
,PC-2011,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PR
,PC,PCH,PCH,PC
,PC,PR,PC,PCH,PC,PC
,PC
,PR,PC,PC
,PR-2011,PC,PM-2001,PC-2008,PR-2006,PC,PC,PC,PC,PC (planes gestion Araba-Bizkaia),PC,PC,PR,PC,PC
,PC-2011,PR,PR,PC,PC,PC,PC,PC
,PC
,PCH
,PC,PCH,PC,PCH,PCH
,PRAH-2012,PR,PR-2007,PR-2004,PR,PR,PR,PR,PR
,PM-1993,PC,PG-2000,PCH,PC
,PR-2011,PR,PC,PCH,PCH,PR,PR
,PCH-2010,PC,PC,PCH,PR,PR-2012,PC
,PC,PCH
,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC,PC
,PR,PR,PR,PR,PR,PR,PR,PR,PR,PR,PR-2013,PR,PR
,PC-2011,PC,PC,PC,PC,PC,PC,PC,PR
,PC-2011,PC,PC,PR,PC,PC,PC,PC,PC,PC
,PCH
,PC,PC,PCH,PCH,PC,PCH
,PM-2002,PR,PC,PCH,PC,PC
,PC,PC,PR
,PC,PCH,PCH,PR
,PC-2011,PR,PR,PR-2008,¿PR?,PR,PR,PR,PR,PR,PR,PR,PR,PR,PR
,PC,PC,PC
,PR,PC,PC,PC,PC,PC,PR
,PC
,PR,PR
,PCH-2002,PC,PC
,PR-2011,PR-2003,PR,PR,PR-1994,PR,PR (PG-2006 -ANULADO),PR-1995
,PC-2011,PR,PR,PC,PC,PC,PR,PR,PC,PC,PR
,PC
,PCH
,PCH-2015,PR,P

/tmp/ipykernel_5034/2516121099.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planes['Ubicacion'][index] = x
/tmp/ipykernel_5034/2516121099.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planes['Plan'][index] = y


,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,Plan
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Islas Baleares",",PC"
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Extremadura",",PCH"
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",País Vasco",",PC"
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Galicia",",PR"
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,",Andalucía,Castilla-La Mancha,Castilla y León,...",",PR-2011,PR-2003,PR-2003,PR-2005,PR"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Extremadura",",PC"
166,Zampullín común,Tachybaptus ruficollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Extremadura,Comunidad Foral de Navarra",",PC,PC"
167,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,",Castilla-La Mancha,Extremadura,Comunidad Fora...",",PC,PCH,PCH"
168,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,


In [48]:
# Quitamos las comas del principio de todos los strings
planes["Ubicacion"].replace({'^\,':''}, regex=True, inplace=True)
planes["Plan"].replace({'^\,':''}, regex=True, inplace=True)
planes

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,Plan
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,PC
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,PCH
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,PC
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,PR
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,"Andalucía,Castilla-La Mancha,Castilla y León,E...","PR-2011,PR-2003,PR-2003,PR-2005,PR"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,PC
166,Zampullín común,Tachybaptus ruficollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,"Extremadura,Comunidad Foral de Navarra","PC,PC"
167,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,"Castilla-La Mancha,Extremadura,Comunidad Foral...","PC,PCH,PCH"
168,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,


In [49]:
# Separamos por un lado el dataset principal con las ubicaciones
planes_ubi = planes.drop(['Plan'], axis=1)
planes_ubi

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,GAL,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,"Andalucía,Castilla-La Mancha,Castilla y León,E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Vencejo real,Tachymarptis melba,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura
166,Zampullín común,Tachybaptus ruficollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,"Extremadura,Comunidad Foral de Navarra"
167,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,"Castilla-La Mancha,Extremadura,Comunidad Foral..."
168,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,


In [50]:
# Y por otro la columna de planes
planes_plan = planes[['Plan']]
planes_plan

,Plan
0,PC
1,PCH
2,PC
3,PR
4,"PR-2011,PR-2003,PR-2003,PR-2005,PR"
...,...
165,PC
166,"PC,PC"
167,"PC,PCH,PCH"
168,


In [51]:
# Dividimos la columna de ubicacion, de esta manera obtendremos un valor de cada ave por cada ubicación en la que tenga un plan
planes_ubica = planes_ubi.assign(Ubicacion=planes_ubi['Ubicacion'].str.split(',')).explode('Ubicacion').reset_index(drop = True)
planes_ubica

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,GAL,MAD,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,PC-2002,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias


#### Hacemos lo mismo con la columna de planes

In [52]:
planes_plan = planes_plan.assign(Plan=planes_plan['Plan'].str.split(',')).explode('Plan').reset_index(drop = True)
planes_plan

,Plan
0,PC
1,PCH
2,PC
3,PR
4,PR-2011
...,...
544,PCH
545,PCH
546,
547,PC-2002


#### Por último hacemos lo mismo con la columna de colores

In [53]:
colores_ccaa_separado = colorsotes_comunidad.assign(HAY_PLAN=colorsotes_comunidad['HAY_PLAN'].str.split(',')).explode('HAY_PLAN').reset_index(drop = True)
colores_ccaa_separado

,HAY_PLAN
0,Rojo
1,Rojo
2,Rojo
3,Rojo
4,Verde
...,...
544,Rojo
545,Rojo
546,
547,Verde


#### Ya tenemos los datos separados en columnas, ahora lo unimos en un solo DF

In [54]:
planes_info = pd.concat([planes_ubica, colores_ccaa_separado, planes_plan], axis=1)
planes_info

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,Rojo,PC
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,Rojo,PC
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,Rojo,PR
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,Verde,PR-2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,Rojo,PCH
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,,
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,PC-2002,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,Verde,PC-2002


## Años de los planes

#### Nos queda separar el año (si lo tiene) de la columna 'Plan', así podremos diferenciar entre planes caducados y planes vigentes.

In [55]:
# Obtenemos los valores únicos de la columna 'Planes'
planes_info['Plan'].unique()

array(['PC', 'PCH', 'PR', 'PR-2011', 'PR-2003', 'PR-2005',
       'PR-2011(Modif. 2013)', 'PRE-2009', 'PR-2016', 'PC-2006',
       'PG-2001', 'PCH-2005', 'PR-2012/PR-2016', 'PR-1996',
       'PR-2009/PR-2016', 'PCAH-2012', 'PC-2007', 'PC-2001', 'PC-2011',
       'PGAE-2014', 'PR-2017', '', 'PM-2001', 'PC-2008', 'PR-2006',
       'PC (planes gestion Araba-Bizkaia)', 'PC-2014', 'PRAH-2012',
       'PR-2007', 'PR-2004', 'PM-1993', 'PG-2000', 'PAC-2005', 'PM-2002',
       'PM-2007', 'PRE-2005-2009', 'PC-2003', 'PCH-2010', 'PR-2012',
       'PR-1995', 'PR-2022', 'PG-2006', 'PAC-2009', 'PR-2013', 'Borrador',
       'PRE-2011', 'PR-2008', '¿PR?', 'PCH-1995', 'PR-2001; PR-2014',
       'PCH-2002', 'PR-1994', 'PR (PG-2006 -ANULADO)',
       'PR-2000; PGAE-2014', 'PR-2014', 'PR-2009', 'PCH-2015', 'PC-2002'],
      dtype=object)

#### En general el año está separado del plan por un guión '-'. Puede servirnos para hacer un split. Vamos a unificar nomenclaturas, para automatizar este proceso.

In [56]:
planes_info[planes_info['Plan']=='PR (PG-2006 -ANULADO)']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
505,Quebrantahuesos,Gypaetus barbatus,2000,PR-2011,PR-2003,PR,NONECESITA,NONECESITA,PR,PR-1994,...,NONECESITA,PR-1995,NONECESITA,NONECESITA,SAP-1997,NONECESITA,NONECESITA,País Vasco,Rojo,PR (PG-2006 -ANULADO)


#### Este dato contiene info de que no hay plan porque fue anulado, solo necesitamos saber que no tiene plan y que plan necesita, así que transformaremos el nombre.

In [57]:
planes_info['Plan'].loc[505]='PR'

In [58]:
planes_info['Plan'][505]

'PR'

In [59]:
planes_info[planes_info['Plan']=='PC (planes gestion Araba-Bizkaia)']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
95,Alimoche común,Neophron percnopterus,Estrategia,PR-2011,PC,PM-2001,PC-2008,PR-2006,PC,PC,...,PC,PC,PC-2014,PC,SAP-2008,NONECESITA,NONECESITA,País Vasco,Rojo,PC (planes gestion Araba-Bizkaia)


#### Aquí ocurre algo parecido, hay info que no necesitamos, modificamos.

In [60]:
planes_info['Plan'].loc[95]='PC'

In [61]:
planes_info['Plan'][95]

'PC'

In [62]:
planes_info[planes_info['Plan']=='PR-2011(Modif. 2013)']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
10,Águila perdicera,Aquila fasciata,Estrategia,PC,PR-2011(Modif. 2013),NONECESITA,PRE-2009,NONECESITA,NONECESITA,PR,...,PR-2012/PR-2016,PR-1996,PR-2009/PR-2016,PC,SAP-1997,NONECESITA,NONECESITA,Aragón,Verde,PR-2011(Modif. 2013)


In [63]:
# Lo mismo
planes_info['Plan'].loc[10]='PR-2011'

In [64]:
planes_info[planes_info['Plan']=='PR-2012/PR-2016']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
19,Águila perdicera,Aquila fasciata,Estrategia,PC,PR-2011(Modif. 2013),NONECESITA,PRE-2009,NONECESITA,NONECESITA,PR,...,PR-2012/PR-2016,PR-1996,PR-2009/PR-2016,PC,SAP-1997,NONECESITA,NONECESITA,Región de Murcia,Verde,PR-2012/PR-2016


#### En este caso nos indican que el plan se actualizó más tarde, nos quedamos con el último año.

In [65]:
planes_info['Plan'].loc[19]='PR-2016'

In [66]:
planes_info[planes_info['Plan']=='PR-2009/PR-2016']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
21,Águila perdicera,Aquila fasciata,Estrategia,PC,PR-2011(Modif. 2013),NONECESITA,PRE-2009,NONECESITA,NONECESITA,PR,...,PR-2012/PR-2016,PR-1996,PR-2009/PR-2016,PC,SAP-1997,NONECESITA,NONECESITA,La Rioja,Verde,PR-2009/PR-2016


In [67]:
# Lo mismo
planes_info['Plan'].loc[21]='PR-2016'

In [68]:
planes_info[planes_info['Plan']=='PRE-2005-2009']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
172,Buitre negro,Aegypius monachus,Estrategia,PC-2011,NONECESITA,NONECESITA,PM-2007,NONECESITA,NONECESITA,PRE-2005-2009,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,Cataluña,Verde,PRE-2005-2009


In [69]:
planes_info['Plan'].loc[172]='PRE-2005'

In [70]:
planes_info[planes_info['Plan']=='Borrador']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
350,Gavilán común,Accipiter nisus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Borrador,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,BMS-1999,NONECESITA,NONECESITA,Islas Canarias,Rojo,Borrador


In [71]:
planes_info['Plan'].loc[350]='PC'

In [72]:
planes_info[planes_info['Plan']=='PR-2000; PGAE-2014']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
522,Sisón común,Tetrax tetrax,2022,PC-2011,PR,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PR,...,PC,PR,PR-2000; PGAE-2014,PAC-2005,SAP-2010,NONECESITA,NONECESITA,La Rioja,Verde,PR-2000; PGAE-2014


In [73]:
planes_info['Plan'].loc[522]='PGAE-2014'

In [74]:

planes_info[planes_info['Plan']=='PR-2001; PR-2014']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
473,Perdiz pardilla,Perdix perdix,Estrategia,NONECESITA,PR,PC,NONECESITA,NONECESITA,PC,PC,...,NONECESITA,PR,PR-2001; PR-2014,NONECESITA,NONECESITA,NONECESITA,NONECESITA,La Rioja,Verde,PR-2001; PR-2014


In [75]:
planes_info['Plan'].loc[473]='PR-2014'

In [76]:
planes_info[planes_info['Plan']=='¿PR?']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,MUR,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan
396,Milano real,Milvus milvus,Estrategia,PC-2011,PR,PR,PR-2008,¿PR?,PR,PR,...,PR,PR,PR,PR,SAP-2009,NONECESITA,NONECESITA,Islas Canarias,Rojo,¿PR?


In [77]:
planes_info['Plan'].loc[396]='PR'

#### Volvemos a comprobar valores unicos

In [78]:
planes_info['Plan'].unique()

array(['PC', 'PCH', 'PR', 'PR-2011', 'PR-2003', 'PR-2005', 'PRE-2009',
       'PR-2016', 'PC-2006', 'PG-2001', 'PCH-2005', 'PR-1996',
       'PCAH-2012', 'PC-2007', 'PC-2001', 'PC-2011', 'PGAE-2014',
       'PR-2017', '', 'PM-2001', 'PC-2008', 'PR-2006', 'PC-2014',
       'PRAH-2012', 'PR-2007', 'PR-2004', 'PM-1993', 'PG-2000',
       'PAC-2005', 'PM-2002', 'PM-2007', 'PRE-2005', 'PC-2003',
       'PCH-2010', 'PR-2012', 'PR-1995', 'PR-2022', 'PG-2006', 'PAC-2009',
       'PR-2013', 'PRE-2011', 'PR-2008', 'PCH-1995', 'PR-2014',
       'PCH-2002', 'PR-1994', 'PR-2009', 'PCH-2015', 'PC-2002'],
      dtype=object)

#### Listo, podemos separar en otra columna los años

In [79]:
planes_con_fecha = planes_info.Plan.str.split(pat='-',expand=True)
planes_con_fecha

,0,1
0,PC,None
1,PCH,None
2,PC,None
3,PR,None
4,PR,2011
...,...,...
544,PCH,None
545,PCH,None
546,,None
547,PC,2002


In [80]:
planes_con_fecha[0].unique()

array(['PC', 'PCH', 'PR', 'PRE', 'PG', 'PCAH', 'PGAE', '', 'PM', 'PRAH',
       'PAC'], dtype=object)

In [81]:
planes_con_fecha[1].unique()

array([None, '2011', '2003', '2005', '2009', '2016', '2006', '2001',
       '1996', '2012', '2007', '2014', '2017', '2008', '2004', '1993',
       '2000', '2002', '2010', '1995', '2022', '2013', '1994', '2015'],
      dtype=object)

In [82]:
planes_final = pd.concat([planes_info, planes_con_fecha], axis=1)
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,Plan,0,1
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,Rojo,PC,PC,None
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,PCH,None
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,Rojo,PC,PC,None
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,Rojo,PR,PR,None
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,Verde,PR-2011,PR,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,PCH,None
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,Rojo,PCH,PCH,None
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,,,,None
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,PC-2002,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,Verde,PC-2002,PC,2002


In [83]:
planes_final.drop(['Plan'], axis=1, inplace=True)
planes_final.rename(columns={0 : 'PLAN', 1 : 'FECHA_PLAN'}, inplace=True)
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,AND,ARA,AST,BAL,CANA,CANT,CAT,...,NAV,RIO,VAL,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,PC,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,Rojo,PC,None
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,None
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,Rojo,PC,None
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,Rojo,PR,None
4,Águila imperial ibérica,Aquila adalberti,2001/2018,PR-2011,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,Verde,PR,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,None
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,PCH,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,Rojo,PCH,None
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,,,None
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,PC-2002,NONECESITA,NONECESITA,NONECESITA,NONECESITA,...,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,Verde,PC,2002


In [84]:
planes_final.drop(['AND', 'ARA', 'AST', 'BAL','CANA', 'CAT', 'NAV', 'RIO', 'VAL', 'CLM', 'CYL', 'EUSK', 'EXT', 'GAL', 'MAD', 'MUR'], axis=1,    inplace=True)
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,Rojo,PC,None
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,None
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,Rojo,PC,None
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,Rojo,PR,None
4,Águila imperial ibérica,Aquila adalberti,2001/2018,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,Verde,PR,2011
...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,Rojo,PCH,None
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,Rojo,PCH,None
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,,,None
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,Verde,PC,2002


In [85]:
planes_final.loc[planes_final['HAY_PLAN']=='Rojo', 'HAY_PLAN'] = 'NO'
planes_final.loc[planes_final['HAY_PLAN']=='Verde', 'HAY_PLAN'] = 'SI'

In [86]:
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,NO,PCH,None
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,NO,PC,None
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,NO,PR,None
4,Águila imperial ibérica,Aquila adalberti,2001/2018,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,SI,PR,2011
...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,NO,PCH,None
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,NO,PCH,None
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,,,,None
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,SI,PC,2002


In [87]:
planes_final.loc[planes_final['Ubicacion']=='', 'Ubicacion'] = 'ESP'
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None
1,Abejero europeo,Pernis apivorus,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,NO,PCH,None
2,Abubilla común,Upupa epops,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,País Vasco,NO,PC,None
3,Agachadiza común,Gallinago gallinago,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Galicia,NO,PR,None
4,Águila imperial ibérica,Aquila adalberti,2001/2018,NONECESITA,SAP-2008,NONECESITA,NONECESITA,Andalucía,SI,PR,2011
...,...,...,...,...,...,...,...,...,...,...,...
544,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,NO,PCH,None
545,Zampullín cuellinegro,Podiceps nigricollis,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Comunidad Foral de Navarra,NO,PCH,None
546,Zarapito fino,Numenius tenuirostris,NONECESITA,NONECESITA,SAP-1996,NONECESITA,NONECESITA,ESP,,,None
547,Zarapito real,Numenius arquata,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2007,Principado de Asturias,SI,PC,2002


## Añadir datos de localización, geopandas

In [88]:
f = r"../ESP/Espana_y_comunidades.shp"
shapes = gpd.read_file(f)

In [89]:
shapes

,index,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,0,215,ESP,Spain,1,Andalucía,Comunidad Autónoma,Autonomous Community,NaN,Andalousie|Andaluc¡a|Andalusien|Andaluzia,"MULTIPOLYGON (((-3.03014 35.94264, -3.03014 35..."
1,1,215,ESP,Spain,2,Aragón,Comunidad Autónoma,Autonomous Community,NaN,Aragão|Aragó|Aragón|Aragona|Aragonien,"POLYGON ((-0.74582 42.92190, -0.74409 42.91921..."
2,2,215,ESP,Spain,3,Cantabria,Comunidad Autónoma,Autonomous Community,NaN,Cantàbria|Cantábria|Cantabrie|Kantabrien,"MULTIPOLYGON (((-3.28365 43.20003, -3.28500 43..."
3,3,215,ESP,Spain,4,Castilla-La Mancha,Comunidad Autónoma,Autonomous Community,NaN,Castela-La Mancha|Castela-Mancha|Castella-la M...,"MULTIPOLYGON (((-3.36418 40.63507, -3.36655 40..."
4,4,215,ESP,Spain,5,Castilla y León,Comunidad Autónoma,Autonomous Community,NaN,Castile and Leon|Castela e Leão|Castella i Lle...,"MULTIPOLYGON (((-2.83047 42.79212, -2.82676 42..."
5,5,215,ESP,Spain,6,Cataluña,Comunidad Autónoma,Autonomous Community,NaN,Catalogna|Catalogne|Catalonia|Catalunha|Catalu...,"MULTIPOLYGON (((0.72958 40.78069, 0.72958 40.7..."
6,6,215,ESP,Spain,7,Ceuta y Melilla,Ciudades Autónomas,Autonomous City,NaN,NaN,"MULTIPOLYGON (((-4.30069 35.17319, -4.30069 35..."
7,7,215,ESP,Spain,8,Comunidad de Madrid,Comunidad Autónoma,Autonomous Community,NaN,Madrid|Communauté de Madrid| Community of Madr...,"MULTIPOLYGON (((-4.25646 40.66382, -4.25743 40..."
8,8,215,ESP,Spain,9,Comunidad Foral de Navarra,Comunidad Autónoma,Autonomous Community,NaN,Communauté forale de Navarre|Comunidade Foral ...,"POLYGON ((-1.65526 43.31251, -1.64447 43.30674..."
9,9,215,ESP,Spain,10,Comunidad Valenciana,Comunidad Autónoma,Autonomous Community,NaN,Valencia|Communauté de Valence|Comunidade Vale...,"MULTIPOLYGON (((-0.45708 38.16208, -0.45708 38..."


In [90]:
ciudades = shapes.drop(['index'], axis=1)
ciudades = ciudades.reset_index()

In [91]:
ciudades

,index,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,0,215,ESP,Spain,1,Andalucía,Comunidad Autónoma,Autonomous Community,NaN,Andalousie|Andaluc¡a|Andalusien|Andaluzia,"MULTIPOLYGON (((-3.03014 35.94264, -3.03014 35..."
1,1,215,ESP,Spain,2,Aragón,Comunidad Autónoma,Autonomous Community,NaN,Aragão|Aragó|Aragón|Aragona|Aragonien,"POLYGON ((-0.74582 42.92190, -0.74409 42.91921..."
2,2,215,ESP,Spain,3,Cantabria,Comunidad Autónoma,Autonomous Community,NaN,Cantàbria|Cantábria|Cantabrie|Kantabrien,"MULTIPOLYGON (((-3.28365 43.20003, -3.28500 43..."
3,3,215,ESP,Spain,4,Castilla-La Mancha,Comunidad Autónoma,Autonomous Community,NaN,Castela-La Mancha|Castela-Mancha|Castella-la M...,"MULTIPOLYGON (((-3.36418 40.63507, -3.36655 40..."
4,4,215,ESP,Spain,5,Castilla y León,Comunidad Autónoma,Autonomous Community,NaN,Castile and Leon|Castela e Leão|Castella i Lle...,"MULTIPOLYGON (((-2.83047 42.79212, -2.82676 42..."
5,5,215,ESP,Spain,6,Cataluña,Comunidad Autónoma,Autonomous Community,NaN,Catalogna|Catalogne|Catalonia|Catalunha|Catalu...,"MULTIPOLYGON (((0.72958 40.78069, 0.72958 40.7..."
6,6,215,ESP,Spain,7,Ceuta y Melilla,Ciudades Autónomas,Autonomous City,NaN,NaN,"MULTIPOLYGON (((-4.30069 35.17319, -4.30069 35..."
7,7,215,ESP,Spain,8,Comunidad de Madrid,Comunidad Autónoma,Autonomous Community,NaN,Madrid|Communauté de Madrid| Community of Madr...,"MULTIPOLYGON (((-4.25646 40.66382, -4.25743 40..."
8,8,215,ESP,Spain,9,Comunidad Foral de Navarra,Comunidad Autónoma,Autonomous Community,NaN,Communauté forale de Navarre|Comunidade Foral ...,"POLYGON ((-1.65526 43.31251, -1.64447 43.30674..."
9,9,215,ESP,Spain,10,Comunidad Valenciana,Comunidad Autónoma,Autonomous Community,NaN,Valencia|Communauté de Valence|Comunidade Vale...,"MULTIPOLYGON (((-0.45708 38.16208, -0.45708 38..."


In [92]:
ciudades = ciudades[['index', 'NAME_1']]
ciudades

,index,NAME_1
0,0,Andalucía
1,1,Aragón
2,2,Cantabria
3,3,Castilla-La Mancha
4,4,Castilla y León
5,5,Cataluña
6,6,Ceuta y Melilla
7,7,Comunidad de Madrid
8,8,Comunidad Foral de Navarra
9,9,Comunidad Valenciana


In [93]:
ciudades.rename(columns={'NAME_1':'Ubicacion'}, inplace=True)
ciudades

/tmp/ipykernel_5034/4123099999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ciudades.rename(columns={'NAME_1':'Ubicacion'}, inplace=True)


,index,Ubicacion
0,0,Andalucía
1,1,Aragón
2,2,Cantabria
3,3,Castilla-La Mancha
4,4,Castilla y León
5,5,Cataluña
6,6,Ceuta y Melilla
7,7,Comunidad de Madrid
8,8,Comunidad Foral de Navarra
9,9,Comunidad Valenciana


In [94]:
planes_final=planes_final.merge(ciudades,how='inner',left_on='Ubicacion',right_on='Ubicacion')
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12
1,Águila perdicera,Aquila fasciata,Estrategia,NONECESITA,SAP-1997,NONECESITA,NONECESITA,Islas Baleares,SI,PRE,2009,12
2,Aguila pescadora,Pandion haliaetus,Estrategia,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2007,12
3,Aguilucho cenizo,Circus pygargus,2022,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12
4,Alimoche común,Neophron percnopterus,Estrategia,PC,SAP-2008,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2008,12
...,...,...,...,...,...,...,...,...,...,...,...,...
544,Pardela sombría,Ardenna grisea,Borrador,NONECESITA,NONECESITA,NONECESITA,NONECESITA,ESP,,,None,18
545,Pico picapinos de Gran Canaria,Dendrocopos major thanneri,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18
546,Pico picapinos de Tenerife,Dendrocopos major canariensis,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18
547,Porrón bastardo,Aythya marila,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2009,ESP,,,None,18


In [95]:
planes_final.rename(columns={'index':'index_ciudad'}, inplace=True)
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index_ciudad
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12
1,Águila perdicera,Aquila fasciata,Estrategia,NONECESITA,SAP-1997,NONECESITA,NONECESITA,Islas Baleares,SI,PRE,2009,12
2,Aguila pescadora,Pandion haliaetus,Estrategia,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2007,12
3,Aguilucho cenizo,Circus pygargus,2022,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12
4,Alimoche común,Neophron percnopterus,Estrategia,PC,SAP-2008,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2008,12
...,...,...,...,...,...,...,...,...,...,...,...,...
544,Pardela sombría,Ardenna grisea,Borrador,NONECESITA,NONECESITA,NONECESITA,NONECESITA,ESP,,,None,18
545,Pico picapinos de Gran Canaria,Dendrocopos major thanneri,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18
546,Pico picapinos de Tenerife,Dendrocopos major canariensis,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18
547,Porrón bastardo,Aythya marila,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2009,ESP,,,None,18


In [96]:
puntos = pd.read_csv('../data_raw/puntos_coordenadas.csv')
puntos

,Unnamed: 0,Ubicacion,lon,lat
0,0,Andalucía,-5.992474,37.386198
1,1,Aragón,-0.879346,41.656446
2,2,Cantabria,-3.804801,43.462914
3,3,Castilla-La Mancha,-4.024317,39.857144
4,4,Castilla y León,-4.723361,41.652293
5,5,Cataluña,2.176246,41.384259
6,6,Ceuta y Melilla,-2.953579,35.295064
7,7,Comunidad de Madrid,-3.687613,40.408399
8,8,Comunidad Foral de Navarra,-1.645226,42.814080
9,9,Comunidad Valenciana,-0.375866,39.471125


In [97]:
puntos.drop(['Unnamed: 0'], axis=1, inplace=True)
puntos

,Ubicacion,lon,lat
0,Andalucía,-5.992474,37.386198
1,Aragón,-0.879346,41.656446
2,Cantabria,-3.804801,43.462914
3,Castilla-La Mancha,-4.024317,39.857144
4,Castilla y León,-4.723361,41.652293
5,Cataluña,2.176246,41.384259
6,Ceuta y Melilla,-2.953579,35.295064
7,Comunidad de Madrid,-3.687613,40.408399
8,Comunidad Foral de Navarra,-1.645226,42.814080
9,Comunidad Valenciana,-0.375866,39.471125


In [98]:
puntos['Ubicacion'].loc[18]='ESP'
puntos

/tmp/ipykernel_5034/32508081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puntos['Ubicacion'].loc[18]='ESP'


,Ubicacion,lon,lat
0,Andalucía,-5.992474,37.386198
1,Aragón,-0.879346,41.656446
2,Cantabria,-3.804801,43.462914
3,Castilla-La Mancha,-4.024317,39.857144
4,Castilla y León,-4.723361,41.652293
5,Cataluña,2.176246,41.384259
6,Ceuta y Melilla,-2.953579,35.295064
7,Comunidad de Madrid,-3.687613,40.408399
8,Comunidad Foral de Navarra,-1.645226,42.814080
9,Comunidad Valenciana,-0.375866,39.471125


In [99]:
planes_final=planes_final.merge(puntos,how='inner',left_on='Ubicacion',right_on='Ubicacion')
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index_ciudad,lon,lat
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12,2.651714,39.571186
1,Águila perdicera,Aquila fasciata,Estrategia,NONECESITA,SAP-1997,NONECESITA,NONECESITA,Islas Baleares,SI,PRE,2009,12,2.651714,39.571186
2,Aguila pescadora,Pandion haliaetus,Estrategia,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2007,12,2.651714,39.571186
3,Aguilucho cenizo,Circus pygargus,2022,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,None,12,2.651714,39.571186
4,Alimoche común,Neophron percnopterus,Estrategia,PC,SAP-2008,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2008,12,2.651714,39.571186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Pardela sombría,Ardenna grisea,Borrador,NONECESITA,NONECESITA,NONECESITA,NONECESITA,ESP,,,None,18,-3.687613,40.408399
545,Pico picapinos de Gran Canaria,Dendrocopos major thanneri,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18,-3.687613,40.408399
546,Pico picapinos de Tenerife,Dendrocopos major canariensis,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,None,18,-3.687613,40.408399
547,Porrón bastardo,Aythya marila,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2009,ESP,,,None,18,-3.687613,40.408399


In [100]:
planes_final['FECHA_PLAN'].fillna(0, inplace=True)

In [101]:
planes_final['FECHA_PLAN']=planes_final['FECHA_PLAN'].astype(str).astype(int)

In [102]:
planes_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 0 to 548
Data columns (total 14 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Nombre común                                         549 non-null    object 
 1   Nombre científico                                    549 non-null    object 
 2   ESTRATEGIA ESTATAL                                   549 non-null    object 
 3   CANT                                                 549 non-null    object 
 4   Planes Acción Comisión Europea                       549 non-null    object 
 5   Planes Acción AEWA                                   549 non-null    object 
 6   Planes Manejo Especies Cinegéticas Comisión Europea  549 non-null    object 
 7   Ubicacion                                            549 non-null    object 
 8   HAY_PLAN                                             549 non-null    o

In [103]:
# pOR ÚLTIMO HACEMOS UNA COLUMNA PARA SABER QUÉ PLANES ESTÁS CADUCADOS
planes_final['Caducado']=np.nan
planes_final.loc[planes_final['FECHA_PLAN']<2018, 'Caducado'] = 'SI'
planes_final.loc[planes_final['FECHA_PLAN']>=2018, 'Caducado'] = 'NO'
planes_final.loc[planes_final['FECHA_PLAN']==0, 'Caducado'] = 'NOPROCEDE'
planes_final

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index_ciudad,lon,lat,Caducado
0,Abejaruco europeo,Merops apiaster,NONECESITA,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,0,12,2.651714,39.571186,NOPROCEDE
1,Águila perdicera,Aquila fasciata,Estrategia,NONECESITA,SAP-1997,NONECESITA,NONECESITA,Islas Baleares,SI,PRE,2009,12,2.651714,39.571186,SI
2,Aguila pescadora,Pandion haliaetus,Estrategia,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2007,12,2.651714,39.571186,SI
3,Aguilucho cenizo,Circus pygargus,2022,PC,NONECESITA,NONECESITA,NONECESITA,Islas Baleares,NO,PC,0,12,2.651714,39.571186,NOPROCEDE
4,Alimoche común,Neophron percnopterus,Estrategia,PC,SAP-2008,NONECESITA,NONECESITA,Islas Baleares,SI,PC,2008,12,2.651714,39.571186,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Pardela sombría,Ardenna grisea,Borrador,NONECESITA,NONECESITA,NONECESITA,NONECESITA,ESP,,,0,18,-3.687613,40.408399,NOPROCEDE
545,Pico picapinos de Gran Canaria,Dendrocopos major thanneri,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,0,18,-3.687613,40.408399,NOPROCEDE
546,Pico picapinos de Tenerife,Dendrocopos major canariensis,NONECESITA,NONECESITA,SAP-1999,NONECESITA,NONECESITA,ESP,,,0,18,-3.687613,40.408399,NOPROCEDE
547,Porrón bastardo,Aythya marila,NONECESITA,NONECESITA,NONECESITA,NONECESITA,EU-MP-2009,ESP,,,0,18,-3.687613,40.408399,NOPROCEDE


In [104]:
planes_final[planes_final['Caducado']=='NO']

,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index_ciudad,lon,lat,Caducado
47,Cigüeña negra,Ciconia nigra,Estrategia,NONECESITA,NONECESITA,NONECESITA,NONECESITA,Extremadura,SI,PR,2022,10,-6.344154,38.917378,NO


In [105]:
# Guardamos
#planes_final.to_csv('../data_raw/planes_aves.csv')

## Mapa de folium para visualizar planes

In [106]:
planes_elegidos = planes_final.copy() # Creamos una copia de nuestra DataFrame, asi no modificamos datos de la original.
planes_elegidos = planes_elegidos[planes_elegidos["HAY_PLAN"] == 'NO']

In [107]:
planes_elegidos['Ubicacion'].unique()

array(['Islas Baleares', 'Extremadura', 'País Vasco', 'Galicia',
       'Andalucía', 'Castilla-La Mancha', 'Castilla y León',
       'Comunidad de Madrid', 'Aragón', 'Cataluña', 'Región de Murcia',
       'Comunidad Foral de Navarra', 'La Rioja', 'Comunidad Valenciana',
       'Principado de Asturias', 'Islas Canarias', 'Cantabria'],
      dtype=object)

In [108]:
import folium
import folium.plugins as plugins
from folium.features import GeoJsonTooltip

In [123]:
#total_ciudades = cuentaTotalPlanes(planes_elegidos)
#print(total_ciudades)
#total = pd.DataFrame({'total': total_ciudades})
#print(total_ciudades)
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)
folium.TileLayer('stamenwatercolor').add_to(mapa)
for index, row in planes_elegidos.iterrows():
        ciudad = row["Ubicacion"]
        latitud = row["lat"]
        longitud = row["lon"]
        misma_ciudad = planes_elegidos[planes_elegidos['Ubicacion']==ciudad]
        aves_ciudad = misma_ciudad['Nombre científico'].to_list()
        text = "<br>".join(aves_ciudad)
        total = len(planes_elegidos[planes_elegidos['Ubicacion']==ciudad])
        #print(ciudad)
        
        # Crear un marcador de burbuja para la ciudad
        burbuja = folium.CircleMarker(location=[latitud, longitud],
                                    radius=total,
                                    color='#3186cc',
                                    fill=True,
                                    fill_color='#3186cc',
                                    tooltip=f"{text}"
                                    )
        
        # Agregar el número de etiqueta encima de la burbuja
        folium.Marker([latitud, longitud], tooltip=f'{ciudad}', icon=plugins.BeautifyIcon(
                            icon="arrow-down", icon_shape="marker",
                            number=total,
                            border_color= '#3186cc',
                            background_color='#3186cc'
                        )
                    ).add_to(mapa)
        
        # Agregar la burbuja al mapa
        burbuja.add_to(mapa)
mapa

In [ ]:
ciu = 'Comunidad de Madrid'
print(len(planes_elegidos[planes_elegidos['Ubicacion']==ciu]))
planes_elegidos[planes_elegidos['Ubicacion']==ciu]

0


,Nombre común,Nombre científico,ESTRATEGIA ESTATAL,CANT,Planes Acción Comisión Europea,Planes Acción AEWA,Planes Manejo Especies Cinegéticas Comisión Europea,Ubicacion,HAY_PLAN,PLAN,FECHA_PLAN,index_ciudad,lon,lat,Caducado
